In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
val_dataset = load_dataset("knkarthick/AMI")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
dataset = pd.read_csv("conversations.csv", encoding='latin1')

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [ ]:
device = 'gpu'
model_ckpt = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

In [ ]:
def get_feature(batch):
  encodings = tokenizer(batch['dialogue'], text_target=batch['summary'],
                        max_length=1024, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)
dataset_pt = dataset.map(get_feature, batched=True)

Map:   0%|          | 0/168 [00:00<?, ? examples/s]

In [ ]:
val_dataset_pt = val_dataset['validation'].map(get_feature, batched=True)

In [ ]:
dataset_pt

Dataset({
    features: ['dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 168
})

In [ ]:
val_dataset_pt

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 42
})

In [ ]:
columns = ['input_ids', 'labels', 'attention_mask']
dataset_pt.set_format(type='torch', columns=columns)

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = 'bart_finetuned',
    num_train_epochs=1,
    warmup_steps = 500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=1
)

trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset = dataset_pt, eval_dataset = val_dataset_pt)



In [ ]:
trainer.train()



You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=168, training_loss=2.866335494177682, metrics={'train_runtime': 108.7205, 'train_samples_per_second': 1.545, 'train_steps_per_second': 1.545, 'total_flos': 328896217006080.0, 'train_loss': 2.866335494177682, 'epoch': 1.0})

In [ ]:
trainer.save_model('bart_finetuned_model')



In [ ]:
pipe = pipeline('summarization', model='bart_finetuned_model')
gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8, "max_length": 120}

custom_dialogue="""
Simran: Interaction in a live section is crucial; we can't see faces online. In class, facial expressions help convey understanding. Now, you can see our faces, but don't give us a logic where you can't immediately counter.

Host: Let's go to Simran.

Simran: Online learning is convenient; it's paperless and eco-friendly. Others may join the discussion.

Radhika: I'm for online learning; it's convenient, flexible, and paper-free.

Just: I disagree; we need social interaction for practical learning. Online has practical issues.

Shruti: Rural areas face internet issues. Zoom has limitations. Lack of internet prevents some from joining online classes.

Bavya: Online learning is beneficial during the pandemic, cost-effective, and comfortable.

Kushi: I'm against online learning; some need traditional methods.

Host: Diverse opinions; let's respect each other's viewpoints. Anyone else want to share?

"""
print(pipe(custom_dialogue, **gen_kwargs))



[{'summary_text': 'The discussion revolved around the benefits and drawbacks of online learning. Simran emphasized the importance of interaction in a live section. Radhika discussed online learning as convenient, flexible, and paper-free. Bavya discussed its benefits during the pandemic, cost-effective, and comfortable. Kushi discussed traditional methods and the need for social interaction. Shruti discussed internet issues, Zoom limitations, and rural areas.'}]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from shutil import copytree
output_dir_local = 'bart_finetuned_model'
output_dir_drive = 'drive/My Drive/bart_finetuned_model'

copytree(output_dir_local, output_dir_drive)

'drive/My Drive/bart_finetuned_model'

In [ ]:
from google.colab import drive
from transformers import BartForConditionalGeneration, BartTokenizer

output_dir_drive = '/content/drive/My Drive/bart_finetuned_model'

# Load the saved model
model = BartForConditionalGeneration.from_pretrained(output_dir_drive)
tokenizer = BartTokenizer.from_pretrained(output_dir_drive)

# Use the loaded model for generation
pipe = pipeline('summarization', model=model, tokenizer=tokenizer)
gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8, "max_length": 120}

custom_dialogue = """
Simran: Interaction in a live section is crucial; we can't see faces online. In class, facial expressions help convey understanding. Now, you can see our faces, but don't give us a logic where you can't immediately counter.

Host: Let's go to Simran.

Simran: Online learning is convenient; it's paperless and eco-friendly. Others may join the discussion.

Radhika: I'm for online learning; it's convenient, flexible, and paper-free.

Just: I disagree; we need social interaction for practical learning. Online has practical issues.

Shruti: Rural areas face internet issues. Zoom has limitations. Lack of internet prevents some from joining online classes.

"""

print(pipe(custom_dialogue, **gen_kwargs))


[{'summary_text': "The discussion revolved around the benefits and drawbacks of online learning. Simran emphasized the importance of live interaction in a live section to convey understanding. Radhika argued for online learning because it's convenient, flexible, paper-free, and flexible. Shruti highlighted internet issues in rural areas and Zoom limitations. Just argued for social interaction for practical learning because of social interaction and practical issues."}]
